#Import, variables globales, classe vm

In [49]:
import math
import numpy
import random
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mean
import sys

In [50]:
tasks = [] # Les tâches à effectuer
vms = [] # Les vms disponibles avec leurs caractèristiques

indiv_length = 500 # nombre de tache

nb_vm_fog = 10
nb_vm_cloud = 3

vm_number = nb_vm_cloud + nb_vm_cloud

min_cost = 0
min_exec_time = 0

mutation_rate = 0.1
iteration_number = 100
population_size = 100

In [51]:
class vm:
  def __init__(self,power,cpu_cost,memory_cost,bandwidth_cost,env) -> None:
    self.power = power # Puissance de la cpu en MIPS
    self.cpu_cost = cpu_cost # Cout d'utilisation de la cpu pour une unité de temps
    self.memory_cost = memory_cost # Cout de la mémoire par MB
    self.bandwidth_cost = bandwidth_cost # Cout de la bande passante par MB
    self.env = env # Environnement du noeud (Fog ou CLoud)

  def show(self):
    print("Power = ",self.power)
    print("Cpu cost = ",self.cpu_cost)
    print("Memory cost = ",self.memory_cost)
    print("Bandwith cost = ",self.bandwidth_cost)
    print("Env = ",self.env,"\n")


class task:
  def __init__(self,nb_instruction, memory,input_size_file, output_size_file) -> None:
    self.nb_instruction = nb_instruction # Nombre d'instruction de la tâche en milliard d'instructions
    self.memory = memory # Mémoire nécessaire en MB
    self.input_size_file = input_size_file # Taille du fichier d'entrée en MB
    self.output_size_file = output_size_file # Taille du fichier de sortie en MB

  def show(self):
    print("Number instruction = ",self.nb_instruction)
    print("Memory = ",self.memory)
    print("Input size file = ",self.input_size_file)
    print("Output size file = ",self.output_size_file,"\n")

# Fonctions générales

## Initialisation

In [52]:
def generate_data(indiv_length) -> None:
  global nb_vm_fog
  global nb_vm_cloud
  global vms
  global tasks
  #global indiv_length
  global vm_number

  tasks = []
  vms = []

  # Création des noeuds fog
  for i in range(0,nb_vm_fog):
    power = numpy.random.uniform(500,1500)
    cpu_cost = numpy.random.uniform(0.1,0.4)
    memory_cost = numpy.random.uniform(0.01,0.03)
    bandwidth_cost = numpy.random.uniform(0.01,0.02)

    vms.append(vm(power,cpu_cost,memory_cost,bandwidth_cost,"fog"))


  # Création des noeuds cloud
  for i in range(0,nb_vm_cloud):
    power = numpy.random.uniform(3000,5000)
    cpu_cost = numpy.random.uniform(0.7,1)
    memory_cost = numpy.random.uniform(0.02,0.05)
    bandwidth_cost = numpy.random.uniform(0.05,0.1)

    vms.append(vm(power,cpu_cost,memory_cost,bandwidth_cost,"cloud"))


  # Création des tâches
  for i in range(0,indiv_length):
    nb_instruction = numpy.random.uniform(1,100)
    memory = numpy.random.uniform(50,200)
    input_size_file = numpy.random.uniform(10,100)
    output_size_file = numpy.random.uniform(10,100)

    tasks.append(task(nb_instruction,memory,input_size_file,output_size_file))


  # Calcul le cout et le temps d'execution minimum
  global min_cost
  global min_exec_time
  min_exec_time = sum([truc.nb_instruction for truc in tasks])*1000/sum([vm.power for vm in vms])
  min_cost = 0
  for truc in tasks:
    tmp = vms[0].cpu_cost*truc.nb_instruction*1000/vms[0].power + vms[0].memory_cost*truc.memory + vms[0].bandwidth_cost*(truc.input_size_file+truc.output_size_file)
    for node in vms:
      val = node.cpu_cost*truc.nb_instruction*1000/node.power + node.memory_cost*truc.memory + node.bandwidth_cost*(truc.input_size_file+truc.output_size_file)
      if val < tmp :
        tmp = val
    min_cost += tmp

In [53]:
# Initialise une population aléatoire

def init_population(indiv_length) -> list:
  population = []
  while len(population) < population_size:
    indiv= numpy.zeros(indiv_length)

    indiv = [random.randint(0,vm_number-1) for elem in indiv]
    if isValid(indiv):
      population.append(indiv)
  return population

In [54]:
# Permet de gérer les contraintes

def isValid(indiv):
  return True

In [55]:
# Permet de calculer la fitness d'une solution entre 0 et 1, fonction à maximiser
def fitness(indiv) -> tuple:

  global min_cost
  global min_exec_time
  global tasks
  global vms
  global vm_number



  cost = 0
  exec_time = numpy.zeros(vm_number)

  for i in range(0,len(indiv)):
    vm = vms[indiv[i]]

    cost += vm.cpu_cost*tasks[i].nb_instruction*1000/vm.power + vm.memory_cost*tasks[i].memory + vm.bandwidth_cost*(tasks[i].input_size_file+tasks[i].output_size_file)
    exec_time[indiv[i]] += tasks[i].nb_instruction*1000/vm.power

  return [min_cost/cost,min_exec_time/max(exec_time)]


# Permet de calculer la valeur des metrics d'une solution

def perf(indiv) -> tuple:

  cost = 0
  exec_time = numpy.zeros(vm_number)


  for i in range(0,len(indiv)):
    vm = vms[indiv[i]]

    cost += vm.cpu_cost*tasks[i].nb_instruction*1000/vm.power + vm.memory_cost*tasks[i].memory + vm.bandwidth_cost*(tasks[i].input_size_file+tasks[i].output_size_file)

    exec_time[indiv[i]] += tasks[i].nb_instruction*1000/vm.power
  return [cost,max(exec_time)]

In [56]:
# Fonction permettant de savoir si un individu en domine un autre

# Prend en entrée la fitness des deux individus
# Renvoie en sortie un tuple de deux éléments le premier disant si l'indiv 1 domine et le deuxième si l'indiv 2 domine.
# Si aucun des deux individus ne se dominent, le tuple renvoyé est (True,True)
# Si les deux individus sont identiques, le tuple renvoyé est (True,True)

def dominance_check(fit1,fit2) -> tuple:
  if fit1 == fit2 :
    return (True,True)
  bool1 = False
  bool2 = False
  for i in range(len(fit1)):
    if fit1[i] > fit2[i]:
      bool1 = True
    elif fit1[i] < fit2[i]:
      bool2 = True
  return (bool1,bool2)


#Fonction permettant d'obtenir les solutions de pareto d'un ensemble

#Prend en entrée une population d'individu
#Renvoie les individus pareto
def pareto_solutions(population) -> list:
  population_fit = list(map(fitness, population))
  pareto = []

  for i in range(len(population)):
    isPareto = True
    for j in range(len(population)):
      domi,_ = dominance_check(population_fit[i],population_fit[j])
      if not domi:
        isPareto = False
        break
    if isPareto:
      pareto.append(population[i])
  return pareto

In [57]:
import pandas as pd

def data_to_csv(indiv_length):

  cost_table = []
  execution_table = []

  for vm in vms:
    tmp_cost = []
    tmp_exec = []
    for task in tasks:
      tmp_cost.append(vm.cpu_cost*task.nb_instruction*1000/vm.power + vm.memory_cost*task.memory + vm.bandwidth_cost*(task.input_size_file+task.output_size_file))
      tmp_exec.append(task.nb_instruction*1000/vm.power)
    cost_table.append(tmp_cost)
    execution_table.append(tmp_exec)
  df_cost = pd.DataFrame(cost_table)
  df_cost.to_csv(f"CostTable_R_{indiv_length}.csv")

  df_exec = pd.DataFrame(execution_table)
  df_exec.to_csv(f"ExecutionTable_R_{indiv_length}.csv")



In [58]:
# fonction main qui s'execute lorsqu'on lance le script

def main(indiv_length):
    generate_data(indiv_length)
    data_to_csv(indiv_length)
    

if __name__ == "__main__":
    main(500)